In [1]:
!pip install web3

In [2]:
import time
import json
import requests
import pandas as pd
from tqdm import tqdm
from web3 import Web3
from web3.exceptions import ABIFunctionNotFound

In [3]:
POLY_API_KEY = "TYVT9VBDKZUHWS35GYYJP3FK7MTS8XS8K9"
POLY_URL = "https://api.polygonscan.com/api"

web3ProviderURL = "https://rpc.ankr.com/polygon/b1bdd2819d3da7be4760d270af7ef985a74725cbe337c348bca633d096806c5a"
w3 = Web3(Web3.HTTPProvider(web3ProviderURL))

getAccountBalance_url = "https://rpc.ankr.com/multichain/79258ce7f7ee046decc3b5292a24eb4bf7c910d7e39b691384c7ce0cfb839a01/?ankr_getAccountBalance="

In [4]:
factories = {
    "sushiswap": {
        "address" : ["0xc35dadb65012ec5796536bd9864ed8773abc74c4"],
        "pool" : []
        }
}

In [5]:
def get_abi(address):
    params = {
        "module": "contract",
        "action": "getabi",
        "address": address,
        "apikey": POLY_API_KEY
    }
    response = requests.post(POLY_URL, params=params)
    time.sleep(0.2)
    data = json.loads(response.text)
    result = data['result']
    return result

In [6]:
def get_contract(address):
    abi = get_abi(address)
    checksum_address = Web3.to_checksum_address(address)
    contract = w3.eth.contract(address=checksum_address, abi=abi)
    return contract

In [7]:
def get_fee_from_fee(contract, name):
    deno_dict = {
        "uniswap_v3" : 10**6,
        "sushiswap_v3" : 10**6,
        "curve" : 10**10,
    }
    if name in deno_dict:
        fee = float(contract.functions.fee().call())
        fee = fee / deno_dict[name]
        return fee
    return None


def get_fee_from_swapFeeUnits(contract, name):
    if name == "kyberswap":
        swapFeeUnits = float(contract.functions.swapFeeUnits().call())
        fee = swapFeeUnits / 100000
        return fee
    return None


def get_fee_from_globalState(contract, name):
    if name == "quickswap_v3":
        globalState = contract.functions.globalState().call()
        fee = globalState[2]
        fee = fee / 1000
        return fee
    return None


def get_fee(contract, name):
    fix_fee_dict = {
        "quickswap": 3/1000,
        "sushiswap" : 3/1000,
        "apeswap" : 2/1000,
        "dfyn" : 3/1000,
        "jetswap" : 1/1000,
    }

    fee_fun = [
        get_fee_from_fee,
        get_fee_from_swapFeeUnits,
        get_fee_from_globalState,
    ]

    if name in fix_fee_dict:
        return fix_fee_dict[name]

    for fun in fee_fun:
        fee = fun(contract, name)
        if fee:
            return fee
    return fee

In [8]:
def get_tokens(contract):
    try:
        token0 = contract.functions.token0().call().lower()
        token1 = contract.functions.token1().call().lower()
        return token0, token1
    except Exception:
        return None, None

In [9]:
def get_pool_from_allPairs(factory, start, end):
    rows = []
    name = factory
    for factory_address in factories[factory]["address"]:
        factory_contract = get_contract(factory_address)
        try:
            allPairsLength = factory_contract.functions.allPairsLength().call()
            if end > allPairsLength:
                end = allPairsLength
            for i in tqdm(range(start, end)):
                try:
                    pool_address = factory_contract.functions.allPairs(i).call().lower()
                    contract = get_contract(pool_address)
                    fee = get_fee(contract, name)
                    token0, token1 = get_tokens(contract)
                    row = {
                        "name" : name,
                        "factory" : factory_address,
                        "fee" : fee,
                        "contract" : pool_address,
                        "token0" : token0,
                        "token1" : token1,
                    }
                    rows.append(row)
                    time.sleep(0.2)
                except ValueError:
                    continue
        except ABIFunctionNotFound:
            continue
    pool = pd.DataFrame(rows)
    pool = pool.drop_duplicates(ignore_index = True)
    return pool

In [10]:
def get_tvl(pool):
    token_type = ['NATIVE']
    rows = []

    for i in tqdm(range(0, len(pool))):
        payload = {
            "jsonrpc": "2.0",
            "method": "ankr_getAccountBalance",
            "params": {
                "blockchain": ["polygon"],
                "walletAddress": pool.contract[i],
                "nativeFirst": True,
                "onlyWhitelisted": True,
                "pageSize": 2
            },
            "id": 1
        }
        headers = {
            "accept": "application/json",
            "content-type": "application/json"
        }

        response = requests.post(getAccountBalance_url, json=payload, headers=headers)
        time.sleep(1)
        if response.status_code == 200:
            try:
                data = json.loads(response.text)
                tvl = data['result']['totalBalanceUsd']
            except KeyError:
                tvl = None
            row = {
                "name" : pool.name[i],
                "factory" : pool.factory[i],
                "tvl" : tvl,
                "fee" : pool.fee[i],
                "contract" : pool.contract[i],
                "token0" : pool.token0[i],
                "token1" : pool.token1[i],
            }

            rows.append(row)
    pool = pd.DataFrame(rows)
    pool = pool.dropna(axis=0).reset_index(drop=True)
    return pool

## for count in range(0, 10000):
    start = count * 1000
    end = start + 1000
    pool = get_pool_from_allPairs("sushiswap", start, end)
    if len(pool) <= 0:
        break
    pool = get_tvl(pool)
    file_name = f"part{count}.csv"
    pool.to_csv(file_name, index=False)

In [12]:
files = [f"part{i}.csv" for i in range(count)]

In [13]:
combined_df = pd.DataFrame()

In [14]:
for file in files:
    df = pd.read_csv(file)
    combined_df = pd.concat([combined_df, df], ignore_index=True)

combined_df = combined_df.drop_duplicates()
combined_df

,name,factory,tvl,fee,contract,token0,token1
0,sushiswap,0xc35dadb65012ec5796536bd9864ed8773abc74c4,3.445596e+06,0.003,0xc4e595acdd7d12fec385e5da5d43160e8a0bac0e,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619
1,sushiswap,0xc35dadb65012ec5796536bd9864ed8773abc74c4,6.073729e+05,0.003,0x34965ba0ac2451a34a0471f04cca3f990b8dea27,0x2791bca1f2de4661ed88a30c99a7a9449aa84174,0x7ceb23fd6bc0add59e62ac25578270cff1b9f619
2,sushiswap,0xc35dadb65012ec5796536bd9864ed8773abc74c4,5.312027e+03,0.003,0x8929d3fea77398f64448c85015633c2d6472fb29,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0x8f3cf7ad23cd3cadbd9735aff958023239c6a063
3,sushiswap,0xc35dadb65012ec5796536bd9864ed8773abc74c4,1.622322e+05,0.003,0xf69e93771f11aecd8e554aa165c3fe7fd811530c,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0x385eeac5cb85a38a9a07a70c73e0a3271cfb54a7
4,sushiswap,0xc35dadb65012ec5796536bd9864ed8773abc74c4,1.459001e+00,0.003,0x3054ed5f568b0ed3d1a506f64291b65da72b97f1,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0x20d3922b4a1a8560e1ac99fba4fade0c849e2142
...,...,...,...,...,...,...,...
7360,sushiswap,0xc35dadb65012ec5796536bd9864ed8773abc74c4,1.000000e-06,0.003,0x232c646544a629a26ffb2b4d05c75560d73e1111,0x592514e7cbdcfc9a2cdadabde573a9b53b909ced,0xc2132d05d31c914a87c6611c10748aeb04b58e8f
7361,sushiswap,0xc35dadb65012ec5796536bd9864ed8773abc74c4,0.000000e+00,0.003,0x736b34e9f4dfcefedaa4638513a5d44ea9b70ea0,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0xc79c2ee7910df7fb7c19ff419bfaf2d1f4617ad0
7362,sushiswap,0xc35dadb65012ec5796536bd9864ed8773abc74c4,0.000000e+00,0.003,0x12cc35bf9d20de2959261c88b750d7ed88961784,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0x267211a4fbb7dc4dd1849357d10c147f233bc3e4
7363,sushiswap,0xc35dadb65012ec5796536bd9864ed8773abc74c4,7.858115e-05,0.003,0x32d8a3a6085543fdc98c1cd996f18500c61d9313,0x0d500b1d8e8ef31e21c99d1db9a6444d3adf1270,0x8d77eb5aae57b9667a3a35cd42da95c4ba93620b


In [15]:
combined_df.to_csv('poly_sushiswap.csv', index=False)